In [1]:
import re
from eval_paths import get_shortname

In [2]:
def Q_to_subscript(s):
    if 'GOODQ' in s:
        match = re.search(r'\d+', s)
        name = r'GOOD\textsubscript{' + match.group(0) + '}'
        return name
    else:
        return s

In [3]:
model_list = [
        r'Plain',
        r'CEDA',
        r'OE',
        r'ACET',
        r'CCU',
        r'GOOD\textsubscript{0}',
        r'GOOD\textsubscript{20}',
        r'GOOD\textsubscript{40}',
        r'GOOD\textsubscript{60}',
        r'GOOD\textsubscript{80}',
        r'GOOD\textsubscript{90}',
        r'GOOD\textsubscript{95}',
        r'GOOD\textsubscript{100}',
]

In [4]:
dset_log_names = {
    'CIFAR-100'      : 'CIFAR100',
    'CIFAR-10'       : 'CIFAR10',
    'SVHN'           : 'SVHN',
    'LSUN Classroom' : 'LSUN',
    'Uniform Noise'  : 'Noise_Dataset_noise_uniform',
    '80M Tiny Images': 'TinyImages',
    'ImageNet-'      : 'ImageNet-',
    'Smooth Noise'   : 'Noise_Dataset_noise_low_freq',
    'Omniglot'       : 'Omniglot',
    'notMNIST'       : 'ImageFolder',
    'FashionMNIST'   : 'FashionMNIST',
    'EMNIST Letters' : 'EMNIST',
}

In [5]:
dset_aauc_names = {
    'CIFAR-100'      : None,
    'CIFAR-10'       : 'GrayCIFAR10',
    'SVHN'           : None,
    'LSUN Classroom' : None,
    'Uniform Noise'  : 'UniformNoise',
    '80M Tiny Images': 'Tiny',
    'ImageNet-'      : None,
    'Smooth Noise'   : None,
    'Omniglot'       : 'Omniglot',
    'notMNIST'       : 'NotMNIST',
    'FashionMNIST'   : 'FMNIST',
    'EMNIST Letters' : 'EMNIST',
}

In [6]:
file_c10 = 'evals/2020-10-14_16:34:38.log'
file_svhn = 'evals/2020-10-14_18:14:40.log'
file_mnist = 'evals/2020-10-15_15:13:11.log'
file_mnist_04 = 'evals/2020-10-16_08:49:04.log'

In [7]:
aauc_file_mnist = 'evals/aauc/MNIST_0.3_2020-10-22 12 47 03.842292.txt'
aauc_file_mnist_04 = 'evals/aauc/MNIST_0.4_2020-10-22 12 49 06.954378.txt'
aauc_file_mnist_add = 'evals/aauc/MNIST_0.3_2020-10-22 12 54 10.248237.txt'

In [8]:
dset_list_C10_main = ['CIFAR-100', 'SVHN', 'LSUN Classroom', 'Uniform Noise']
dset_list_C10_add = ['80M Tiny Images', 'ImageNet-', 'Smooth Noise']
dset_list_svhn_main = ['CIFAR-100', 'CIFAR-10', 'LSUN Classroom', 'Uniform Noise']
dset_list_svhn_add = ['80M Tiny Images', 'ImageNet-', 'Smooth Noise']
dset_list_mnist_main = ['FashionMNIST', 'EMNIST Letters', 'CIFAR-10', 'Uniform Noise']
dset_list_mnist_add = ['80M Tiny Images', 'Omniglot', 'notMNIST']

In [9]:
def tex_length(c):
    c = c.strip()
    if len(c) == 4:
        c = r'\noindent\phantom{0}' + c
    elif len(c) == 3:
        c = r'\noindent\phantom{00}' + c
    return c.ljust(17)

In [10]:
def get_table(logfile_dest, model_list, dset_list):
    from_log = {}
    dset_pos_in_log = {}
    got_datasets = False
    logfile = open(logfile_dest)
    for l in logfile:
        if 'Model & Acc. &' in l and not got_datasets:
            got_datasets = True
            for i, d in enumerate(l.split('&')):
                if d[-3:] == ' P ':
                    dset_pos_in_log[d[1:-3]] = i
        if '.pt &' in l:
            from_log[Q_to_subscript(get_shortname(l))] = l
    for name in model_list:
        if name not in from_log.keys():
            row = []
            row += [name, 'X']
            for d in dset_list:
                row += 3*['X']
        else:
            values = from_log[name][:-3].split('&')
            row = [] 
            row += [name, values[1]]
            for d in dset_list:
                p = dset_pos_in_log[dset_log_names[d]]
                row += values[p:p+3]
        tex_row = row[:1] + [tex_length(e) for e in row[1:]]
        print(' & '.join(tex_row) + ' \\\\')

In [11]:
def name_from_aauc_line(l):
    f = l.split('&')[0].strip()
    if f in {'Plain', 'CEDA', 'OE', 'ACET'}:
        return f
    elif f[:4] == 'GOOD':
        match = re.search(r'\d+', f)
        name = r'GOOD\textsubscript{' + match.group(0) + r'}'
        return name
    else:
        return None

In [12]:
def get_table_aauc(logfile_dest, aaucfile_dest, model_list, dset_list):
    from_log = {}
    dset_pos_in_log = {}
    got_datasets = False
    logfile = open(logfile_dest)
    for l in logfile:
        if 'Model & Acc. &' in l and not got_datasets:
            got_datasets = True
            for i, d in enumerate(l.split('&')):
                if d[-3:] == ' P ':
                    dset_pos_in_log[d[1:-3]] = i
        if '.pt &' in l:
            from_log[Q_to_subscript(get_shortname(l))] = l
    aaucfile = open(aaucfile_dest)
    dset_pos_in_aauc = {}
    aaucs = {}
    for l in aaucfile:
        if '{} & ' in l:
            for i, d in enumerate(l[:-3].split('&')):
                for n in dset_aauc_names.keys():
                    if dset_aauc_names[n] == d.strip():
                        dset_pos_in_aauc[n] = i
        lname = name_from_aauc_line(l)
        if lname not in model_list:
            continue
        else:
            aaucs[lname] = {}
            for d in dset_pos_in_aauc.keys():
                aaucs[lname][d] = l[:-3].split('&')[dset_pos_in_aauc[d]].strip()
    for name in model_list:
        if name not in from_log.keys():
            row = []
            row += [name, 'X']
            for d in dset_list:
                row += 3*['X']
        else:
            values = from_log[name][:-3].split('&')
            row = [] 
            row += [name, values[1]]
            for d in dset_list:
                p = dset_pos_in_log[dset_log_names[d]]
                row.append(values[p])
                if name not in aaucs.keys():
                    row.append('A')
                elif d in aaucs[name].keys():
                    row.append(aaucs[name][d])
                else:
                    row.append('A')
                row.append(values[p+2])
        tex_row = row[:1] + [tex_length(e) for e in row[1:]]
        print(' & '.join(tex_row) + ' \\\\')

In [13]:
get_table_aauc(file_mnist, aauc_file_mnist, model_list, dset_list_mnist_main)

Plain & \noindent\phantom{0}99.4 & \noindent\phantom{0}98.0 & \noindent\phantom{0}34.2 & \noindent\phantom{00}0.0 & \noindent\phantom{0}88.0 & \noindent\phantom{0}31.4 & \noindent\phantom{00}0.0 & \noindent\phantom{0}98.8 & \noindent\phantom{0}36.6 & \noindent\phantom{00}0.0 & \noindent\phantom{0}99.1 & \noindent\phantom{0}36.5 & \noindent\phantom{00}0.0 \\
CEDA & \noindent\phantom{0}99.4 & \noindent\phantom{0}99.9 & \noindent\phantom{0}82.1 & \noindent\phantom{00}0.0 & \noindent\phantom{0}92.6 & \noindent\phantom{0}52.8 & \noindent\phantom{00}0.0 & 100.0             & \noindent\phantom{0}95.1 & \noindent\phantom{00}0.0 & 100.0             & 100.0             & \noindent\phantom{00}0.0 \\
OE & \noindent\phantom{0}99.4 & \noindent\phantom{0}99.9 & \noindent\phantom{0}76.8 & \noindent\phantom{00}0.0 & \noindent\phantom{0}92.7 & \noindent\phantom{0}50.9 & \noindent\phantom{00}0.0 & 100.0             & \noindent\phantom{0}92.4 & \noindent\phantom{00}0.0 & 100.0             & 100.0         

In [14]:
get_table_aauc(file_mnist, aauc_file_mnist_add, model_list, dset_list_mnist_add)

Plain & \noindent\phantom{0}99.4 & \noindent\phantom{0}98.7 & \noindent\phantom{0}36.9 & \noindent\phantom{00}0.0 & \noindent\phantom{0}97.9 & \noindent\phantom{0}38.6 & \noindent\phantom{00}0.0 & \noindent\phantom{0}91.9 & \noindent\phantom{0}38.8 & \noindent\phantom{00}0.0 \\
CEDA & \noindent\phantom{0}99.4 & 100.0             & \noindent\phantom{0}94.3 & \noindent\phantom{00}0.0 & \noindent\phantom{0}98.5 & \noindent\phantom{0}53.1 & \noindent\phantom{00}0.0 & \noindent\phantom{0}99.9 & \noindent\phantom{0}97.8 & \noindent\phantom{00}0.0 \\
OE & \noindent\phantom{0}99.4 & 100.0             & \noindent\phantom{0}91.5 & \noindent\phantom{00}0.0 & \noindent\phantom{0}98.5 & \noindent\phantom{0}51.0 & \noindent\phantom{00}0.0 & \noindent\phantom{0}99.9 & \noindent\phantom{0}96.8 & \noindent\phantom{00}0.0 \\
ACET & \noindent\phantom{0}99.4 & 100.0             & \noindent\phantom{0}99.2 & \noindent\phantom{00}0.0 & \noindent\phantom{0}99.5 & \noindent\phantom{0}76.5 & \noindent\phantom{0

In [15]:
get_table_aauc(file_mnist_04, aauc_file_mnist_04, model_list, dset_list_mnist_main)

Plain & \noindent\phantom{0}99.4 & \noindent\phantom{0}98.0 & \noindent\phantom{0}28.6 & \noindent\phantom{00}0.0 & \noindent\phantom{0}88.0 & \noindent\phantom{0}26.9 & \noindent\phantom{00}0.0 & \noindent\phantom{0}98.8 & \noindent\phantom{0}32.4 & \noindent\phantom{00}0.0 & \noindent\phantom{0}99.2 & \noindent\phantom{0}34.3 & \noindent\phantom{00}0.0 \\
CEDA & \noindent\phantom{0}99.4 & \noindent\phantom{0}99.9 & \noindent\phantom{0}69.9 & \noindent\phantom{00}0.0 & \noindent\phantom{0}92.6 & \noindent\phantom{0}49.1 & \noindent\phantom{00}0.0 & 100.0             & \noindent\phantom{0}82.8 & \noindent\phantom{00}0.0 & 100.0             & 100.0             & \noindent\phantom{00}0.0 \\
OE & \noindent\phantom{0}99.4 & \noindent\phantom{0}99.9 & \noindent\phantom{0}63.6 & \noindent\phantom{00}0.0 & \noindent\phantom{0}92.7 & \noindent\phantom{0}47.4 & \noindent\phantom{00}0.0 & 100.0             & \noindent\phantom{0}76.0 & \noindent\phantom{00}0.0 & 100.0             & \noindent\phan